In [14]:
## Assignment 2 ##

import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import scipy
from scipy import misc
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
A = genfromtxt('adult_data.csv', delimiter=',')
B = genfromtxt('adult_data.csv', delimiter=',',dtype=str)
#np.savetxt('Resized_Images111.csv', resized_images, delimiter=',')

features = A[:,[0,2,4,10,11,12]]
labels = B[:,14]

features_scaled = preprocessing.scale(features)


In [6]:
np.shape(features)
#np.shape(labels)


(48842, 6)

In [33]:
# Removing missing examples from feature 3 (missing examples => elements with value = 0)
#features = features[np.nonzero(features[:,3])]

# Removing missing examples from feature  (missing examples => elements with value = 0)
for i in range(len(labels)):
    if labels[i] == '<=50k':
        labels[i] = -1
    if labels[i] == '>50k':
        labels[i] = 1
#features = features[np.nonzero(features[:,4])]
temp[temp == ' <=50k'] = -1
temp[temp == '>50k'] = 1
#temp = labels.tolist()
#print(type(temp[0]))
type(temp)

list

In [17]:
td,test_features,tl,test_labels= train_test_split(features_scaled, labels, test_size=.1)
train_features, validated_features,train_labels, validated_labels = train_test_split(td,tl,test_size=int(.1*len(labels)))
test_labels

array([' <=50K', ' >50K', ' <=50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype='<U27')

In [8]:
#Creating Feature Sets
np.random.shuffle(features_scaled)
x_training, x_test, x_val = features_scaled[:int(len(features_scaled)*0.8),:], features_scaled[int(0.8*len(features_scaled)):int(0.9*len(features_scaled)),:], features_scaled[int(0.9*len(features_scaled)):int(1.0*len(features_scaled)),:]

#Creating Label Sets
np.random.shuffle(labels)
y_training, y_test, y_val = labels[:int(len(labels)*0.8)], labels[int(0.8*len(labels)):int(0.9*len(labels))], labels[int(0.9*len(labels)):int(1.0*len(labels))]

#Returning 'features_scaled' and 'labels' to original form
features_scaled = A[:,[0,2,4,10,11,12]]
labels = B[:,14]

In [9]:
# Defining Variables
reg_cons = [0.001, 0.01, 0.1, 1.0]
epochs = 50
steps = 300

In [13]:
np.isnan(features_scaled).any()

False